#### Context Question Answering

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nastyachizhikova/doc_test/blob/main/source/notebooks/squad.ipynb)

# Table of contents 

1. [Introduction to the task](#1.-Introduction-to-the-task)

2. [Get started with the model](#2.-Get-started-with-the-model)

3. [Use the model for prediction](#3.-Use-the-model-for-prediction)

    3.1. [Predict using Python](#3.1-Predict-using-Python)
    
    3.2. [Predict using Python pipeline](#3.2-Predict-using-Python-pipeline)
    
    3.3. [Predict using CLI](#3.3-Predict-using-CLI)
     
4. [Train the model on your data](#4.-Train-the-model-on-your-data)
    
    4.1. [Train your model from Python](#4.1-Train-your-model-from-Python)
    
    4.2. [Train your model from CLI](#4.2-Train-your-model-from-CLI)
    
5. [Models list](#5.-Models-list)

# 1. Introduction to the task

Context Question Answering is a task of finding a fragment with an answer to a question in a given segment of context.

**Context**:

In meteorology, precipitation is any product of the condensation of atmospheric water vapor that falls under gravity. The main forms of precipitation include drizzle, rain, sleet, snow, graupel and hail… Precipitation forms as smaller droplets coalesce via collision with other rain drops or ice crystals **within a cloud**. Short, intense periods of rain in scattered locations are called “showers”.

**Question**:
Where do water droplets collide with ice crystals to form precipitation?

**Answer**: within a cloud

Datasets that follow this task format:

- [Stanford Question Answering Dataset (SQuAD) (EN)](https://rajpurkar.github.io/SQuAD-explorer/)

- [SberQuAD (RU)](https://paperswithcode.com/dataset/sberquad)

To learn more about Context Question Answering application, read [our article](https://medium.com/deeppavlov/developing-qa-systems-for-any-language-with-deeppavlov-a9033d5231a8) on Medium. 

# 2. Get started with the model

First make sure you have the DeepPavlov Library installed.
[More info about the first installation](https://deeppavlov-test.readthedocs.io/en/latest/notebooks/Get%20Started%20with%20DeepPavlov.html)

In [ ]:
!pip install --q deeppavlov

Then make sure that all the required packages for the model are installed.

In [ ]:
!python -m deeppavlov install squad_torch_bert

`squad_torch_bert` here is the name of the model's *config_file*. [What is a Config File?](https://deeppavlov-test.readthedocs.io/en/latest/notebooks/Config%20File.html) 

Configuration file defines the model and describes its hyperparameters. To use another model, change the name of the *config_file* here and further.
The full list of the models with their config names can be found in the [table](#4.-Models-list).


# 3. Use the model for prediction

## 3.1 Predict using Python

After [installing](#2.-Get-started-with-the-model) the model, build it from the config and predict.

In [ ]:
from deeppavlov import configs, build_model

model = build_model(configs.squad.squad_torch_bert, download=True)

**Input**: List[context, question]

**Output**: List[answer, start_character, logit]

In [4]:
model(['DeepPavlov is a library for NLP and dialog systems.'], ['What is DeepPavlov?'])

[['a library for NLP and dialog systems'], [14], [200928.390625]]

## 3.2 Predict using Python pipeline

Alternatively, you can use a Python way to describe and build your model for prediction, without using the config file.

In [ ]:
from pathlib import Path

from deeppavlov import Element, Model
from deeppavlov.download import download_resource
from deeppavlov.models.preprocessors.torch_transformers_preprocessor import TorchSquadTransformersPreprocessor
from deeppavlov.models.torch_bert.torch_transformers_sequence_tagger import TorchTransformersSequenceTagger
from deeppavlov.models.preprocessors.squad_preprocessor import SquadBertAnsPreprocessor, SquadBertMappingPreprocessor, SquadBertAnsPostprocessor
from deeppavlov.models.torch_bert.torch_transformers_squad import TorchTransformersSquad


transformer = 'bert-base-uncased'
model_path = Path('./squad_torch_bert')
lowercase = True 

download_resource(
    'http://files.deeppavlov.ai/v1/squad/squad_torch_bert.tar.gz',
    {'./squad_torch_bert'}
)

in_preprocessor = TorchSquadTransformersPreprocessor(
    vocab_file=transformer,
    do_lower_case=lowercase,
    max_seq_length=384,
    return_tokens=True
)
 
mapping = SquadBertMappingPreprocessor(
    do_lower_case=lowercase
)

transformer = TorchTransformersSquad(
    pretrained_bert=transformer,
    save_path=model_path/'model',
    load_path=model_path/'model',
    optimizer='AdamW',
    optimizer_parameters={
        'lr': 2e-05,
        'weight_decay': 0.01,
        'betas': [
            0.9,
            0.999
        ],
        'eps': 1e-06
    },
    learning_rate_drop_patience=2,
    learning_rate_drop_div=2.0,
)

ans_postprocessor = SquadBertAnsPostprocessor()

model = Model(
    x=['context_raw', 'question_raw'],
    out=['ans_predicted', 'ans_start_predicted',
      "logits"],
    pipe=[
        Element(component=in_preprocessor, 
                x=['context_raw', 'question_raw'], 
                out=['bert_features', 'subtokens']),
          
        Element(component=mapping, 
                x=['context_raw', 'bert_features', 'subtokens'], 
                out=['subtok2chars', 'char2subtoks']),
          
        Element(component=transformer, 
                x=['bert_features'], 
                out=['ans_start_predicted', 'ans_end_predicted', 'logits']),
        
        Element(component=ans_postprocessor, 
                x=['ans_start_predicted', 'ans_end_predicted', 'context_raw', 'bert_features', 'subtok2chars', 'subtokens'], 
                out=['ans_predicted', 'ans_start_predicted', 'ans_end_predicted'])
    ]
)

In [ ]:
model(['DeepPavlov is a library for NLP and dialog systems.'], ['What is DeepPavlov?'])

[['a library for NLP and dialog systems'], [14], [200928.390625]]

## 3.3 Predict using CLI

You can also get predictions in an interactive mode through CLI.

In [ ]:
!python -m deeppavlov interact squad_torch_bert -d

`-d` is an optional download key (alternative to `download=True` in Python code). The key `-d` is used to download the pre-trained model along with embeddings and all other files needed to run the model. 

Or make predictions for samples from *stdin*.

In [ ]:
!python -m deeppavlov predict squad_torch_bert -f <file-name>

# 4. Train the model on your data


## 4.1 Train your model from Python

### Provide your data path

To train the model on your data, you need to change the path to the training data in the *config_file*. 

Parse the *config_file* and change the path to your data from Python.

In [ ]:
from deeppavlov import configs, train_model
from deeppavlov.core.commands.utils import parse_config

model_config = parse_config(configs.squad.squad_torch_bert)

#  dataset that the model was trained on
print(model_config['dataset_reader']['data_path'])

~/.deeppavlov/downloads/squad/


Provide a *data_path* to your own dataset. 

In [ ]:
# download and unzip a new example dataset
!wget http://files.deeppavlov.ai/datasets/squad-v1.1.tar.gz
!tar -xzvf "squad-v1.1.tar.gz"

Note that if you want to provide your own dataset, it should have the same format as the SQuAD dataset downloaded in this cell.

In [ ]:
# provide a path to the train file
model_config["dataset_reader"]["data_path"] = "/contents/train-v1.1.json"


### Train the model using new config

In [ ]:
model = train_model(model_config)

Use your model for prediction.

In [ ]:
model(['DeepPavlov is a library for NLP and dialog systems.'], ['What is DeepPavlov?'])

[['a library for NLP and dialog systems'], [14], [200928.390625]]

## 4.2 Train your model from CLI

In [ ]:
!python -m deeppavlov train squad_torch_bert

# 5. Models list

The table presents a list of all of the NER-models available in DeepPavlov Library.

| Config name  | Dataset | Language | Model Size | F1 score |
| :--- | --- | --- | --- | ---: |
| [ner_ontonotes_bert](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/configs/ner/ner_ontonotes_bert.json) | Ontonotes | En | 1.3 GB | 87.9 |
| [ner_ontonotes_bert_probas](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/configs/ner/ner_ontonotes_bert_probas.json)| ? | ? |
| [ner_ontonotes_bert_mult](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/configs/ner/ner_ontonotes_bert_mult.json)| Ontonotes | Multi | 2.0 GB | 87.2 |
| [ner_rus_bert](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/configs/ner/ner_rus_bert.json)| Collection3   | Ru | 2.0 GB | 97.7 |
| [ner_rus_bert_probas](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/configs/ner/ner_rus_bert_probas.json)| Collection3   | Ru | ? | ? |
| [ner_rus_convers_distilrubert_2L](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/configs/ner/ner_rus_convers_distilrubert_2L.json)| Collection3   | Ru | ? | ? |
| [ner_rus_convers_distilrubert_6L](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/configs/ner/ner_rus_convers_distilrubert_6L.json)| Collection3   | Ru |? | ? |